In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import shutil
import time

from api import *
from dataframe import *
from database import *
from file import *
from sql import *
from tables import *

# cfg

In [2]:
cfg = {
    "host": f"192.168.0.144",
    "user": f"{os.environ['POSTGRE_USER']}",
    "password": f"{os.environ['POSTGRE_PASSWORD']}",
    "port": "5432",
    "database": "herbario"
}
filename = "original.csv"
filename_george = "csv/dados-george.csv"

engine, session = connect(cfg)

2022-11-15 19:09:31,604 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-11-15 19:09:31,605 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-11-15 19:09:31,606 INFO sqlalchemy.engine.Engine select current_schema()
2022-11-15 19:09:31,607 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-11-15 19:09:31,608 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-11-15 19:09:31,608 INFO sqlalchemy.engine.Engine [raw sql] {}


# Load data

In [3]:
dataframe = pandas.read_csv("csv/original.csv", sep=";", low_memory=False, skipinitialspace=True)
dataframe_george = pandas.read_csv(filename_george, sep=";", low_memory=False, skipinitialspace=True)

In [ ]:
create_table_if_not_exists(cfg, engine, "county")
create_table_if_not_exists(cfg, engine, "data")

In [ ]:
if session.query(County).count() == 0:
    response = get_municipies()
    for i, county in enumerate(response.json()):
        session.add(create_county(county))
        make_operation(session)

In [ ]:
if session.query(DataSP).count() == 0:
    change_header(dataframe)
    dataframe = preprocess(dataframe)
    # dataframe.iloc[24230]
    for row in dataframe.iterrows():
        session.add(create_datasp(row[1]))
        make_operation(session)

In [ ]:
if session.query(DataSP).filter(DataSP.george == True).count() == 0:
    for row in dataframe_george.iterrows():
        if row[1]["GEORGE"].lower() == "sim":
            session.query(DataSP).filter(DataSP.seq == row[1]["seq"]).update({"george": True}, synchronize_session=False)
            make_operation(session)

In [4]:
from unaccent import unaccent

data_piperaceae = session.query(DataSP).all()
data_uf = session.query(sqlalchemy.func.lower(unaccent(County.uf))).distinct().all()
data_uf_name = session.query(County.uf_name, sqlalchemy.func.lower(unaccent(County.uf_name))).distinct().all()
data_county = session.query(County.county, unaccent(County.county)).filter(County.uf=="PR").all()

2022-11-15 19:09:36,471 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-15 19:09:36,474 INFO sqlalchemy.engine.Engine SELECT data.seq AS data_seq, data.modified AS data_modified, data.institution_code AS data_institution_code, data.collection_code AS data_collection_code, data.catalog_number AS data_catalog_number, data.basis_of_record AS data_basis_of_record, data.kingdom AS data_kingdom, data.phylum AS data_phylum, data.classe AS data_classe, data."order" AS data_order, data.family AS data_family, data.genus AS data_genus, data.specific_epithet AS data_specific_epithet, data.infraspecific_epithet AS data_infraspecific_epithet, data.scientific_name AS data_scientific_name, data.scientific_name_authorship AS data_scientific_name_authorship, data.identified_by AS data_identified_by, data.year_identified AS data_year_identified, data.month_identified AS data_month_identified, data.day_identified AS data_day_identified, data.type_status AS data_type_status, data.recorded_by AS data

/tmp/ipykernel_99727/3996990519.py:4: SAWarning: Class unaccent will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  data_uf = session.query(sqlalchemy.func.lower(unaccent(County.uf))).distinct().all()


### create file meta-data imgs george

In [ ]:
result = session.query(DataSP.barcode, DataSP.genus, DataSP.specific_epithet).filter(DataSP.george==True).all()

columns = ['barcode', 'genus', 'specific_epithet']
values = [row for row in result]
output = pd.DataFrame(values, columns=columns)
output.to_excel('genus-and-specific_epithet-by-image.xlsx', na_rep='', engine='xlsxwriter', index=None)
output.to_csv('genus-and-specific_epithet-by-image.csv', na_rep='')

In [5]:
weight = create_tsvector(
    (DataSP.county, 'A'),
    (DataSP.state_province, 'B'),
    (DataSP.country, 'C')
)
session.query(DataSP).filter().update(values=dict({"index": weight}), synchronize_session=False)
make_operation(session)

query = sqlalchemy.func.to_tsquery("portuguese", "Paraná")
session.query(DataSP.seq, sqlalchemy.func.ts_rank(DataSP.index, query)).filter(DataSP.index.op("@@")(query)).all()

2022-11-15 19:15:53,765 INFO sqlalchemy.engine.Engine UPDATE data SET index=((setweight(to_tsvector(%(to_tsvector_1)s, data.county), %(setweight_1)s) || setweight(to_tsvector(%(to_tsvector_2)s, data.state_province), %(setweight_2)s)) || setweight(to_tsvector(%(to_tsvector_3)s, data.country), %(setweight_3)s))
2022-11-15 19:15:53,766 INFO sqlalchemy.engine.Engine [generated in 0.00064s] {'to_tsvector_1': 'portuguese', 'setweight_1': 'A', 'to_tsvector_2': 'portuguese', 'setweight_2': 'B', 'to_tsvector_3': 'portuguese', 'setweight_3': 'C'}
2022-11-15 19:15:54,152 INFO sqlalchemy.engine.Engine COMMIT
2022-11-15 19:15:54,639 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-15 19:15:54,641 INFO sqlalchemy.engine.Engine SELECT data.seq AS data_seq, ts_rank(data.index, to_tsquery(%(to_tsquery_1)s, %(to_tsquery_2)s)) AS ts_rank_1 
FROM data 
WHERE data.index @@ to_tsquery(%(to_tsquery_1)s, %(to_tsquery_2)s)
2022-11-15 19:15:54,641 INFO sqlalchemy.engine.Engine [generated in 0.00066s] {'to

[(105, 0.24317084),
 (203, 0.66871977),
 (4, 0.24317084),
 (5, 0.24317084),
 (14, 0.24317084),
 (15, 0.24317084),
 (16, 0.24317084),
 (17, 0.24317084),
 (18, 0.24317084),
 (19, 0.24317084),
 (20, 0.24317084),
 (21, 0.24317084),
 (22, 0.24317084),
 (23, 0.24317084),
 (24, 0.24317084),
 (25, 0.24317084),
 (26, 0.24317084),
 (27, 0.24317084),
 (28, 0.24317084),
 (29, 0.24317084),
 (30, 0.24317084),
 (31, 0.24317084),
 (32, 0.24317084),
 (34, 0.24317084),
 (35, 0.24317084),
 (36, 0.24317084),
 (9849, 0.24317084),
 (37, 0.24317084),
 (9866, 0.24317084),
 (9906, 0.24317084),
 (38, 0.24317084),
 (9947, 0.24317084),
 (9959, 0.24317084),
 (9976, 0.24317084),
 (39, 0.24317084),
 (10017, 0.24317084),
 (40, 0.24317084),
 (10055, 0.24317084),
 (10075, 0.24317084),
 (41, 0.24317084),
 (10111, 0.24317084),
 (10131, 0.24317084),
 (42, 0.24317084),
 (43, 0.24317084),
 (44, 0.24317084),
 (10229, 0.24317084),
 (45, 0.24317084),
 (46, 0.24317084),
 (47, 0.24317084),
 (48, 0.24317084),
 (49, 0.24317084),
 

In [ ]:
session.close()
engine.dispose()